## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "Rocket Classic"
tournament_date = "6/29/2025"  # Ending date of tournament
old_course = "Detroit Golf Club"
tournament_id = "R2025524"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "John Deere Classic"
new_ending_date = "7/6/2025"
new_course = "TPC Deere Run"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025524 (Rocket Classic), year: 20250
✅ 156 new rows added for 'Rocket Classic'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
6,2025,2025-06-29,R2025524,Rocket Classic,Detroit Golf Club,Aldrich Potgieter,1,1,-10,-2,-7,-3,"$1,728,000.00",500.000
36,2025,2025-06-29,R2025524,Rocket Classic,Detroit Golf Club,Chris Kirk,T2,2,-7,-7,-3,-5,"$854,400.00",245.000
98,2025,2025-06-29,R2025524,Rocket Classic,Detroit Golf Club,Max Greyserman,T2,2,-9,-2,-6,-5,"$854,400.00",245.000
67,2025,2025-06-29,R2025524,Rocket Classic,Detroit Golf Club,Jake Knapp,T4,4,E,-11,-6,-4,"$432,000.00",122.500
103,2025,2025-06-29,R2025524,Rocket Classic,Detroit Golf Club,Michael Thorbjornsen,T4,4,-7,-5,-4,-5,"$432,000.00",122.500


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,618,.1909,2025
1,Aaron Baddeley,167.0,-1.150,170.0,-1.243,152.0,-0.369,8.0,0.463,153.0,...,289.7,106.0,58.07%,168.0,60.51%,79.0,60.46%,483,.2741,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331,.4285,2025
3,Aaron Rai,26.0,0.769,21.0,0.407,46.0,0.280,66.0,0.082,114.0,...,287.3,1.0,74.04%,21.0,69.44%,114.0,58.77%,34,2.6617,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,837,.1142,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 8 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
619,2016,TPC Deere Run,030,John Deere Classic,2016-08-14
464,2017,TPC Deere Run,030,John Deere Classic,2017-07-16
309,2018,TPC Deere Run,030,John Deere Classic,2018-07-15
154,2019,TPC Deere Run,030,John Deere Classic,2019-07-14
0,2021,TPC Deere Run,030,John Deere Classic,2021-07-11
930,2022,TPC Deere Run,030,John Deere Classic,2022-07-03
1086,2023,TPC Deere Run,R2023030,John Deere Classic,2023-07-09
1242,2024,TPC Deere Run,R2024030,John Deere Classic,2024-07-07


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-08-14 — John Deere Classic (516 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,17,10,357.0,58.8,21.00,2,2016-08-14,John Deere Classic
1,Aaron Wise,1,0,0.0,0.0,0.00,0,2016-08-14,John Deere Classic
2,Abraham Ancer,9,3,55.5,33.3,6.17,1,2016-08-14,John Deere Classic



📆 2017-07-16 — John Deere Classic (455 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,7,228.29,46.7,15.22,0,2017-07-16,John Deere Classic
1,Aaron Rai,1,0,0.00,0.0,0.00,0,2017-07-16,John Deere Classic
2,Aaron Wise,6,4,0.00,66.7,0.00,4,2017-07-16,John Deere Classic



📆 2018-07-15 — John Deere Classic (458 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-07-03,John Deere Classic
1,Aaron Baddeley,4,1,39.46,25.0,9.86,0,2022-07-03,John Deere Classic
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-07-03,John Deere Classic



📆 2023-07-09 — John Deere Classic (541 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,10,325.286,66.7,21.69,0,2023-07-09,John Deere Classic
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-07-09,John Deere Classic
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-07-09,John Deere Classic



📆 2024-07-07 — John Deere Classic (510 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,10,184.725,71.4,13.19,1,2024-07-07,John Deere Classic
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-07-07,John Deere Classic
2,Aaron Rai,17,14,728.257,82.4,42.84,8,2024-07-07,John Deere Classic


### Recent Form

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-08-14 — John Deere Classic (516 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jared du Toit,1,9.0,12.98,2016-08-14,John Deere Classic
1,Tyrrell Hatton,3,10.7,7.72,2016-08-14,John Deere Classic
2,Matthew Southgate,1,12.0,17.31,2016-08-14,John Deere Classic



📆 2017-07-16 — John Deere Classic (455 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Keith Mitchell,1,11.0,15.87,2017-07-16,John Deere Classic
1,Oscar Fraustro,1,13.0,18.76,2017-07-16,John Deere Classic
2,Cam Davis,1,15.0,21.64,2017-07-16,John Deere Classic



📆 2018-07-15 — John Deere Classic (458 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Haotong Li,1,12.0,17.31,2022-07-03,John Deere Classic
1,Martin Contini,1,16.0,23.08,2022-07-03,John Deere Classic
2,Grant Hirschman,1,17.0,24.53,2022-07-03,John Deere Classic



📆 2023-07-09 — John Deere Classic (541 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Joohyung Kim,1,1.0,1.44,2023-07-09,John Deere Classic
1,Brooks Koepka,3,6.7,4.83,2023-07-09,John Deere Classic
2,Scottie Scheffler,18,7.6,2.58,2023-07-09,John Deere Classic



📆 2024-07-07 — John Deere Classic (510 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,3,3.0,2.16,2024-07-07,John Deere Classic
1,Scottie Scheffler,16,6.3,2.22,2024-07-07,John Deere Classic
2,Xander Schauffele,16,12.9,4.55,2024-07-07,John Deere Classic


### Course History

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for John Deere Classic on 2016-08-14


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2016-08-14,TPC Deere Run,John Deere Classic
1,Adam Hadwin,1,18.0,25.97,2016-08-14,TPC Deere Run,John Deere Classic
2,Alex Cejka,1,55.0,79.35,2016-08-14,TPC Deere Run,John Deere Classic



🏌️‍♂️ Course history for John Deere Classic on 2017-07-16


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2017-07-16,TPC Deere Run,John Deere Classic
1,Aaron Wise,1,16.0,23.08,2017-07-16,TPC Deere Run,John Deere Classic
2,Abraham Ancer,1,27.0,38.95,2017-07-16,TPC Deere Run,John Deere Classic



🏌️‍♂️ Course history for John Deere Classic on 2018-07-15


...

,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,90.0,64.92,2022-07-03,TPC Deere Run,John Deere Classic
1,Aaron Wise,3,58.3,42.05,2022-07-03,TPC Deere Run,John Deere Classic
2,Abraham Ancer,1,27.0,38.95,2022-07-03,TPC Deere Run,John Deere Classic



🏌️‍♂️ Course history for John Deere Classic on 2023-07-09


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,77.0,55.54,2023-07-09,TPC Deere Run,John Deere Classic
1,Aaron Wise,3,58.3,42.05,2023-07-09,TPC Deere Run,John Deere Classic
2,Abraham Ancer,1,27.0,38.95,2023-07-09,TPC Deere Run,John Deere Classic



🏌️‍♂️ Course history for John Deere Classic on 2024-07-07


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,4,65.5,40.70,2024-07-07,TPC Deere Run,John Deere Classic
1,Aaron Wise,2,79.5,72.36,2024-07-07,TPC Deere Run,John Deere Classic
2,Adam Hadwin,2,90.0,81.92,2024-07-07,TPC Deere Run,John Deere Classic


In [12]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

## Training Dataset

In [9]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-14,John Deere Classic,TPC Deere Run,Aaron Wise,T16,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,609.0,0.26,80.0,0.0,0.00,0.00,0.0,90.0,129.84,NaN,NaN,1
1,2016,2016-08-14,John Deere Classic,TPC Deere Run,Abraham Ancer,T27,27,144.0,-0.307,111.0,-0.024,161.0,-0.340,77.0,0.057,172.0,-0.421,154.0,18.63,120.0,3.08,136.0,4.07,183.0,4.80,131.0,205.0,178.0,276.4,27.0,65.42,174.0,61.81,131.0,56.67,619.0,0.25,NaN,33.3,55.50,6.17,1.0,76.3,33.14,NaN,NaN,0
2,2016,2016-08-14,John Deere Classic,TPC Deere Run,Adam Hadwin,T8,8,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,80.0,72.2,452.17,25.12,0.0,51.4,17.46,18.0,25.97,1
3,2016,2016-08-14,John Deere Classic,TPC Deere Run,Alex Prugh,T52,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,544.0,0.30,NaN,25.0,0.68,0.17,0.0,88.8,55.17,28.0,40.40,0
4,2016,2016-08-14,John Deere Classic,TPC Deere Run,Andres Gonzales,T64,64,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,NaN,35.3,95.30,5.61,1.0,77.7,26.88,90.0,129.84,0
5,2016,2016-08-14,John Deere Classic,TPC Deere Run,Andres Romero,T67,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,525.0,0.32,NaN,16.7,9.00,1.50,0.0,85.0,43.68,NaN,NaN,0
6,2016,2016-08-14,John Deere Classic,TPC Deere Run,Andrew Landry,T8,8,172.0,-0.694,11.0,0.594,181.0,-0.892,179.0,-0.396,4.0,0.654,160.0,18.48,136.0,3.09,34.0,4.02,169.0,4.76,5.0,85.0,59.0,294.7,26.0,65.55,117.0,64.63,28.0,61.86,482.0,0.37,NaN,40.0,79.29,7.93,0.0,75.1,31.32,NaN,NaN,1
7,2016,2016-08-14,John Deere Classic,TPC Deere Run,Andrew Loupe,T16,16,150.0,-0.395,105.0,-0.005,154.0,-0.281,158.0,-0.172,38.0,0.280,12.0,22.74,78.0,3.06,177.0,4.11,13.0,4.58,101.0,189.0,5.0,309.3,184.0,45.56,148.0,63.41,175.0,52.83,194.0,0.91,NaN,47.1,381.50,22.44,0.0,65.4,22.63,90.0,129.84,1
8,2016,2016-08-14,John Deere Classic,TPC Deere Run,Arjun Atwal,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,887.0,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,129.84,0
9,2016,2016-08-14,John Deere Classic,TPC Deere Run,Armando Villarreal,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [10]:
# Play with different years to see if I notice anything wrong (check Odds)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
1087,2024,2024-07-07,John Deere Classic,TPC Deere Run,Aaron Rai,T7,7,9.0,1.131,36.0,0.303,8.0,0.676,56.0,0.152,83.0,0.054,99.0,22.10%,38.0,3.01,10.0,3.97,44.0,4.54,53.0,153.0,152.0,293.8,1.0,72.02%,7.0,71.58%,33.0,63.18%,21.0,3.2294,20.0,82.4,728.257,42.84,8.0,37.5,12.97,NaN,NaN,1
1088,2024,2024-07-07,John Deere Classic,TPC Deere Run,Adam Long,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,548.0,0.2354,400.0,50.0,125.756,12.58,0.0,62.0,25.86,44.8,27.84,0
1089,2024,2024-07-07,John Deere Classic,TPC Deere Run,Adam Schenk,CUT,90,146.0,-0.421,53.0,0.212,166.0,-0.520,130.0,-0.114,85.0,0.051,132.0,21.24%,152.0,3.09,131.0,4.04,116.0,4.61,102.0,180.0,82.0,302.4,98.0,59.93%,149.0,64.50%,74.0,60.50%,82.0,1.4461,60.0,62.5,577.709,24.07,0.0,54.3,16.87,38.8,21.65,0
1090,2024,2024-07-07,John Deere Classic,TPC Deere Run,Adam Svensson,T34,34,33.0,0.592,99.0,0.019,28.0,0.380,39.0,0.193,164.0,-0.453,153.0,20.41%,142.0,3.08,85.0,4.01,121.0,4.62,62.0,158.0,118.0,298.3,40.0,64.18%,88.0,67.13%,60.0,61.39%,112.0,1.1671,40.0,76.9,487.341,18.74,0.0,48.1,14.59,21.0,15.15,0
1091,2024,2024-07-07,John Deere Classic,TPC Deere Run,Adrien Dumont de Chassart,T46,46,169.0,-1.094,157.0,-0.381,157.0,-0.421,157.0,-0.292,141.0,-0.244,141.0,21.06%,142.0,3.08,141.0,4.05,116.0,4.61,154.0,229.0,69.0,303.9,160.0,54.73%,104.0,66.23%,171.0,53.47%,263.0,0.5384,250.0,41.7,37.255,3.10,0.0,77.3,30.14,NaN,NaN,0


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [11]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-14,John Deere Classic,TPC Deere Run,Aaron Wise,T16,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,609.0,0.26,80.0,0.0,0.00,0.00,0.0,90.0,129.84,NaN,NaN,1
1,2016,2016-08-14,John Deere Classic,TPC Deere Run,Abraham Ancer,T27,27,144.0,-0.307,111.0,-0.024,161.0,-0.340,77.0,0.057,172.0,-0.421,154.0,18.63,120.0,3.08,136.0,4.07,183.0,4.80,131.0,205.0,178.0,276.4,27.0,65.42,174.0,61.81,131.0,56.67,619.0,0.25,1000.0,33.3,55.50,6.17,1.0,76.3,33.14,NaN,NaN,0
2,2016,2016-08-14,John Deere Classic,TPC Deere Run,Adam Hadwin,T8,8,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,80.0,72.2,452.17,25.12,0.0,51.4,17.46,18.0,25.97,1
3,2016,2016-08-14,John Deere Classic,TPC Deere Run,Alex Prugh,T52,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,544.0,0.30,1000.0,25.0,0.68,0.17,0.0,88.8,55.17,28.0,40.40,0
4,2016,2016-08-14,John Deere Classic,TPC Deere Run,Andres Gonzales,T64,64,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,1000.0,35.3,95.30,5.61,1.0,77.7,26.88,90.0,129.84,0


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [12]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-14,John Deere Classic,TPC Deere Run,Aaron Wise,T16,16,115.182432,-0.123625,108.475225,-0.05639,110.260135,-0.049856,105.672297,-0.017413,102.983108,-0.018832,107.084459,20.731847,94.333333,3.060248,98.507312,4.037075,102.759009,4.648041,106.115991,200.498874,106.519144,295.220721,93.97973,60.796498,97.56243,66.136862,101.846847,58.683874,609.0,0.26,80.0,0.0,0.00,0.00,0.0,90.0,129.84,59.058171,59.776011,1
1,2016,2016-08-14,John Deere Classic,TPC Deere Run,Abraham Ancer,T27,27,144.000000,-0.307000,111.000000,-0.02400,161.000000,-0.340000,77.000000,0.057000,172.000000,-0.421000,154.000000,18.630000,120.000000,3.080000,136.000000,4.070000,183.000000,4.800000,131.000000,205.000000,178.000000,276.400000,27.00000,65.420000,174.00000,61.810000,131.000000,56.670000,619.0,0.25,1000.0,33.3,55.50,6.17,1.0,76.3,33.14,59.058171,59.776011,0
2,2016,2016-08-14,John Deere Classic,TPC Deere Run,Adam Hadwin,T8,8,111.000000,-0.011000,55.000000,0.22300,121.000000,-0.055000,132.000000,-0.081000,11.000000,0.530000,22.000000,21.740000,16.000000,3.010000,48.000000,4.030000,68.000000,4.660000,35.000000,141.000000,92.000000,290.100000,49.00000,63.620000,83.00000,66.070000,97.000000,58.270000,193.0,0.91,80.0,72.2,452.17,25.12,0.0,51.4,17.46,18.000000,25.970000,1
3,2016,2016-08-14,John Deere Classic,TPC Deere Run,Alex Prugh,T52,52,115.182432,-0.123625,108.475225,-0.05639,110.260135,-0.049856,105.672297,-0.017413,102.983108,-0.018832,107.084459,20.731847,94.333333,3.060248,98.507312,4.037075,102.759009,4.648041,106.115991,200.498874,106.519144,295.220721,93.97973,60.796498,97.56243,66.136862,101.846847,58.683874,544.0,0.30,1000.0,25.0,0.68,0.17,0.0,88.8,55.17,28.000000,40.400000,0
4,2016,2016-08-14,John Deere Classic,TPC Deere Run,Andres Gonzales,T64,64,160.000000,-0.469000,141.000000,-0.22200,134.000000,-0.112000,151.000000,-0.135000,94.000000,0.007000,91.000000,20.010000,78.000000,3.060000,162.000000,4.090000,80.000000,4.670000,145.000000,224.000000,59.000000,294.700000,165.00000,54.550000,81.00000,66.170000,148.000000,55.790000,270.0,0.66,1000.0,35.3,95.30,5.61,1.0,77.7,26.88,90.000000,129.840000,0


In [13]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [14]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk.head(10)

,PLAYER,SALARY
0,Jason Day,10500
1,Ben Griffin,10400
2,J.T. Poston,10100
3,Denny McCarthy,9900
4,Si Woo Kim,9700
5,Davis Thompson,9600
6,Jake Knapp,9400
7,Chris Gotterup,9300
8,Sungjae Im,9200
9,Michael Thorbjornsen,9100


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,John Deere Classic,Ben Griffin,14/1,14.0
2,2025,John Deere Classic,Denny McCarthy,25/1,25.0
3,2025,John Deere Classic,Jason Day,25/1,25.0
4,2025,John Deere Classic,Davis Thompson,30/1,30.0
5,2025,John Deere Classic,J.T. Poston,30/1,30.0
6,2025,John Deere Classic,Si Woo Kim,30/1,30.0
7,2025,John Deere Classic,Sungjae Im,35/1,35.0
8,2025,John Deere Classic,Jake Knapp,35/1,35.0
9,2025,John Deere Classic,Chris Kirk,35/1,35.0
10,2025,John Deere Classic,Kevin Yu,35/1,35.0


## Cut Percentage and FedEx Points

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-07-06,John Deere Classic
1,Aaron Baddeley,16,6,49.088,37.5,3.07,0,2025-07-06,John Deere Classic
2,Aaron Rai,17,13,660.480,76.5,38.85,2,2025-07-06,John Deere Classic
3,Aaron Wise,2,1,6.333,50.0,3.17,1,2025-07-06,John Deere Classic
4,Adam Hadwin,19,13,214.432,68.4,11.29,4,2025-07-06,John Deere Classic
5,Adam Long,1,0,0.000,0.0,0.00,0,2025-07-06,John Deere Classic
6,Adam Schenk,20,9,270.904,45.0,13.55,0,2025-07-06,John Deere Classic
7,Adam Scott,13,11,392.203,84.6,30.17,6,2025-07-06,John Deere Classic
8,Adam Svensson,20,10,126.043,50.0,6.30,0,2025-07-06,John Deere Classic
9,Adrien Dumont de Chassart,7,3,200.500,42.9,28.64,0,2025-07-06,John Deere Classic


## Recent Form

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Carlos Ortiz,1,4.0,5.77,2025-07-06,John Deere Classic
1,Scottie Scheffler,15,6.9,2.49,2025-07-06,John Deere Classic
2,Jon Rahm,3,9.7,7.00,2025-07-06,John Deere Classic
3,Bubba Watson,1,14.0,20.20,2025-07-06,John Deere Classic
4,Ian Gilligan,1,16.0,23.08,2025-07-06,John Deere Classic
5,Taisei Shimizu,1,16.0,23.08,2025-07-06,John Deere Classic
6,Stewart Cink,1,17.0,24.53,2025-07-06,John Deere Classic
7,Rory McIlroy,11,19.0,7.65,2025-07-06,John Deere Classic
8,Ren Yonezawa,1,22.0,31.74,2025-07-06,John Deere Classic
9,Yuto Katsuragawa,1,22.0,31.74,2025-07-06,John Deere Classic


## Course History

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,4,65.5,40.70,2025-07-06,TPC Deere Run,John Deere Classic
1,Aaron Rai,1,7.0,10.10,2025-07-06,TPC Deere Run,John Deere Classic
2,Aaron Wise,2,79.5,72.36,2025-07-06,TPC Deere Run,John Deere Classic
3,Adam Hadwin,2,90.0,81.92,2025-07-06,TPC Deere Run,John Deere Classic
4,Adam Long,5,53.8,30.03,2025-07-06,TPC Deere Run,John Deere Classic
5,Adam Schenk,6,47.3,24.31,2025-07-06,TPC Deere Run,John Deere Classic
6,Adam Svensson,4,24.3,15.10,2025-07-06,TPC Deere Run,John Deere Classic
7,Adrien Dumont de Chassart,1,46.0,66.36,2025-07-06,TPC Deere Run,John Deere Classic
8,Akshay Bhatia,1,35.0,50.49,2025-07-06,TPC Deere Run,John Deere Classic
9,Alejandro Tosti,1,90.0,129.84,2025-07-06,TPC Deere Run,John Deere Classic


## Merged Dataframe

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Wise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6200
1,Adam Hadwin,149.0,-0.578,138.0,-0.250,154.0,-0.387,73.0,0.059,109.0,-0.069,68.0,22.13%,39.0,3.03,147.0,4.06,128.0,4.65,96.0,175,104.0,298.7,71.0,60.80%,134.0,64.12%,160.0,54.07%,117.0,1.1817,2025.0,200.0,68.4,214.432,11.29,4.0,61.7,20.60,90.0,81.92,6800
2,Adam Schenk,114.0,-0.086,80.0,0.103,119.0,-0.108,118.0,-0.081,115.0,-0.097,127.0,20.69%,39.0,3.03,156.0,4.08,47.0,4.57,143.0,216,59.0,304.8,157.0,53.06%,158.0,61.82%,97.0,59.44%,158.0,.8602,2025.0,150.0,45.0,270.904,13.55,0.0,66.6,21.88,47.3,24.31,6800
3,Adam Svensson,74.0,0.235,118.0,-0.077,73.0,0.110,37.0,0.202,162.0,-0.598,155.0,19.61%,94.0,3.07,114.0,4.04,74.0,4.60,99.0,178,151.0,293.9,27.0,64.91%,49.0,67.32%,108.0,59.00%,208.0,.6781,2025.0,150.0,50.0,126.043,6.30,0.0,67.4,22.14,24.3,15.10,6700
4,Aldrich Potgieter,72.0,0.239,4.0,0.671,110.0,-0.062,160.0,-0.371,29.0,0.275,24.0,23.50%,80.0,3.06,75.0,4.02,132.0,4.66,39.0,143,1.0,327.4,142.0,54.83%,93.0,65.53%,163.0,53.31%,49.0,2.2116,2025.0,45.0,41.7,941.000,78.42,2.0,58.4,22.77,NaN,NaN,7900


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [21]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [22]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [23]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [24]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Wise,98.906542,-0.03272,91.654206,-0.007916,96.607477,-0.023374,89.579439,-0.001411,93.448598,-0.034738,97.719626,21.325794,80.672897,3.061963,84.046729,4.026916,83.859813,4.608598,86.364486,172.018692,88.186916,301.195327,83.831776,59.979439,90.495327,65.687664,77.401869,60.489346,1000.0,4.0421,2025.0,1000.0,52.478358,0.000,18.485821,1.380597,90.0,24.382313,54.928866,49.376392,6200
1,Adam Hadwin,149.000000,-0.57800,138.000000,-0.250000,154.000000,-0.387000,73.000000,0.059000,109.000000,-0.069000,68.000000,22.130000,39.000000,3.030000,147.000000,4.060000,128.000000,4.650000,96.000000,175.000000,104.000000,298.700000,71.000000,60.800000,134.000000,64.120000,160.000000,54.070000,117.0,1.1817,2025.0,200.0,68.400000,214.432,11.290000,4.000000,61.7,20.600000,90.000000,81.920000,6800
2,Adam Schenk,114.000000,-0.08600,80.000000,0.103000,119.000000,-0.108000,118.000000,-0.081000,115.000000,-0.097000,127.000000,20.690000,39.000000,3.030000,156.000000,4.080000,47.000000,4.570000,143.000000,216.000000,59.000000,304.800000,157.000000,53.060000,158.000000,61.820000,97.000000,59.440000,158.0,0.8602,2025.0,150.0,45.000000,270.904,13.550000,0.000000,66.6,21.880000,47.300000,24.310000,6800
3,Adam Svensson,74.000000,0.23500,118.000000,-0.077000,73.000000,0.110000,37.000000,0.202000,162.000000,-0.598000,155.000000,19.610000,94.000000,3.070000,114.000000,4.040000,74.000000,4.600000,99.000000,178.000000,151.000000,293.900000,27.000000,64.910000,49.000000,67.320000,108.000000,59.000000,208.0,0.6781,2025.0,150.0,50.000000,126.043,6.300000,0.000000,67.4,22.140000,24.300000,15.100000,6700
4,Aldrich Potgieter,72.000000,0.23900,4.000000,0.671000,110.000000,-0.062000,160.000000,-0.371000,29.000000,0.275000,24.000000,23.500000,80.000000,3.060000,75.000000,4.020000,132.000000,4.660000,39.000000,143.000000,1.000000,327.400000,142.000000,54.830000,93.000000,65.530000,163.000000,53.310000,49.0,2.2116,2025.0,45.0,41.700000,941.000,78.420000,2.000000,58.4,22.770000,54.928866,49.376392,7900
5,Alejandro Tosti,50.000000,0.47100,30.000000,0.345000,125.000000,-0.175000,18.000000,0.301000,166.000000,-0.688000,60.000000,22.220000,106.000000,3.080000,54.000000,4.010000,63.000000,4.590000,29.000000,131.000000,13.000000,313.300000,118.000000,56.950000,123.000000,64.700000,77.000000,60.660000,149.0,0.9114,2025.0,150.0,42.100000,316.783,16.670000,0.000000,66.4,22.160000,90.000000,129.840000,6800
6,Alex Smalley,9.000000,0.98500,19.000000,0.454000,60.000000,0.201000,16.000000,0.329000,49.000000,0.197000,20.000000,23.710000,80.000000,3.060000,2.000000,3.940000,63.000000,4.590000,3.000000,76.000000,47.000000,306.000000,29.000000,64.820000,32.000000,68.750000,20.000000,64.130000,119.0,1.1687,2025.0,45.0,66.700000,639.388,30.450000,0.000000,45.9,14.850000,38.800000,24.110000,7900
7,Anders Albertson,98.906542,-0.03272,91.654206,-0.007916,96.607477,-0.023374,89.579439,-0.001411,93.448598,-0.034738,97.719626,21.325794,80.672897,3.061963,84.046729,4.026916,83.859813,4.608598,86.364486,172.018692,88.186916,301.195327,83.831776,59.979439,90.495327,65.687664,77.401869,60.489346,1000.0,4.0421,2025.0,1000.0,52.478358,0.000,18.485821,1.380597,90.0,24.382313,54.928866,49.376392,6000
8,Andrew Putnam,77.000000,0.22400,164.000000,-0.650000,36.000000,0.377000,6.000000,0.497000,13.000000,0.517000,95.000000,21.380000,13.000000,3.000000,8.000000,3.970000,38.000000,4.560000,96.000000,175.000000,169.000000,282.200000,6.000000,70.110000,11.000000,70.230000,2.000000,68.660000,124.0,1.1315,2025.0,65.0,64.700000,441.783,25.990000,4.000000,47.2,16

## Correlations
**Last 4 Years**

In [25]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [27]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.95759291 0.94111524 0.94822393 0.92601205 0.92510139]
Average ROC AUC score: 0.94


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [30]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Pierceson Coody,8400,0.904765,62.2,30.000000,35.0,45.5,122.0,161.875,98.906542,91.654206,96.607477,89.579439,93.448598,97.719626,80.672897,84.046729,83.859813,86.364486,88.186916,83.831776,90.495327,77.401869
1,Ryan Gerard,8500,0.873345,35.6,26.000000,35.0,94.7,71.0,825.212,22.000000,47.000000,22.000000,67.000000,117.000000,92.000000,39.000000,54.000000,128.000000,35.000000,84.000000,55.000000,105.000000,73.000000
2,Mark Hubbard,7700,0.865003,54.1,23.000000,50.0,63.6,113.0,535.946,99.000000,97.000000,88.000000,121.000000,81.000000,60.000000,64.000000,75.000000,56.000000,34.000000,85.000000,52.000000,81.000000,91.000000
3,Thriston Lawrence,7300,0.855245,72.6,54.928866,80.0,27.3,77.0,177.043,98.906542,91.654206,96.607477,89.579439,93.448598,97.719626,80.672897,84.046729,83.859813,86.364486,88.186916,83.831776,90.495327,77.401869
4,Henrik Norlander,7100,0.801162,52.8,67.300000,125.0,65.0,180.0,384.129,57.000000,134.000000,5.000000,147.000000,94.000000,99.000000,29.000000,21.000000,92.000000,66.000000,133.000000,26.000000,36.000000,29.000000
5,Zach Johnson,6900,0.716355,63.3,34.700000,125.0,50.0,186.0,288.896,98.906542,91.654206,96.607477,89.579439,93.448598,97.719626,80.672897,84.046729,83.859813,86.364486,88.186916,83.831776,90.495327,77.401869
6,Bud Cauley,8600,0.714204,33.6,39.500000,40.0,87.5,59.0,957.882,22.000000,50.000000,16.000000,86.000000,43.000000,79.000000,80.000000,54.000000,74.000000,103.000000,78.000000,101.000000,97.000000,41.000000
7,Jason Day,10500,0.707190,28.2,23.000000,25.0,92.3,28.0,1070.313,44.000000,91.000000,77.000000,14.000000,77.000000,55.000000,64.000000,21.000000,47.000000,157.000000,131.000000,101.000000,77.000000,44.000000
8,Ryo Hisatsune,7400,0.698824,47.6,52.000000,80.0,71.4,90.0,574.119,37.000000,90.000000,53.000000,27.000000,85.000000,139.000000,10.000000,35.000000,63.000000,139.000000,120.000000,91.000000,35.000000,9.000000
9,Kevin Roy,7400,0.697693,50.5,31.000000,65.0,61.5,152.0,287.321,43.000000,43.000000,50.000000,98.000000,89.000000,60.000000,39.000000,5.000000,38.000000,3.000000,45.000000,31.000000,16.000000,24.000000


### Save to CSV

In [31]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
